In [4]:
import fitz
import string
import pandas as pd
from difflib import SequenceMatcher

In [5]:
filepath_1to5 = r'REP-EDC-2020_Fusion_Final-1-5.pdf'
filepath_full = r'REP-EDC-2020_Fusion_Final.pdf'

In [9]:
def openPDFasTextDict(filepath):
    """
    Opens PDF as XML dict
    """
    text_dict = []
    with fitz.open(filepath) as doc:
        for page in doc:
            text_dict.append(page.get_text("dict", sort=False))
    return text_dict

In [10]:
def removePuncandSpace(text):
    """
    Removes punctuation and spaces from a string
    """
    return text.translate(str.maketrans('', '', string.punctuation)).strip()

In [11]:
removePuncandSpace("yo!")

'yo'

In [12]:
def extractFromTextDict(text_dict):
    """
    Extracts desired text from a text dictionary
    Returns list of dicts
    """
    org_list = []
    org_id = -1
    org_name = False

    for count_page, page in enumerate(text_dict):
        for count_block_list, block_list in enumerate(page["blocks"]):
            for count_line_list, line_list in enumerate(block_list["lines"]):
                for count_spans_list, spans_list in enumerate(line_list["spans"]):

                    #Remove empty text
                    if spans_list['text'].isspace():
                        continue

                    #Check if font & size are that of org number or new org
                    if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                        #Check if start of new org
                        try :
                            #Throws ValueError if name of org
                            int(spans_list['text'])
                        except ValueError:
                            #Only triggers when name of org
                            org_list[org_id]['Name'] = spans_list['text'].strip()                   
                        else:
                            #If not name of org then org number
                            if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                                org_list.append({'id' : spans_list['text'].strip()})   
                                org_id += 1

                    #Check if font & size are that of org address
                    #Uses round to filter more text: other text has size that rounds to 8
                    if (spans_list['font'] == 'Helvetica') & (round(float(spans_list['size'])) == 9):
                        #Catch if no orgs created
                        if org_id < 0:
                            continue
                        #If key Address doesn't already exist, create it
                        if 'Address' not in org_list[org_id].keys():
                            org_list[org_id]['Address'] = ''
                            org_list[org_id]['Address'] += spans_list['text']
                            org_address = True
                        else:
                            #Strip here to avoid unnecessary blank space
                            #Maybe handle this later?
                            org_list[org_id]['Address'] += spans_list['text'].strip()

                    #Check if font & size are that of field name
                    if (spans_list['font'] == 'ArialNarrow') & (int(float(spans_list['size'])) == 8):
                        #Catch if no orgs created
                        if org_id < 0:
                            continue
                        #If key field doesn't already exist, create it. Checks if length string > 1 to remove bad text
                        if (removePuncandSpace(spans_list['text']) not in org_list[org_id].keys()) & (len(removePuncandSpace(spans_list['text'])) > 1):
                            org_list[org_id][removePuncandSpace(spans_list['text'])] = ''

                    #Check if font & size are that of field text
                    if (spans_list['font'] == 'Helvetica-Bold') & (round(float(spans_list['size'])) == 8):
                        #Catch if no orgs created
                        if org_id < 0:
                            continue

                        #Place in last dict key: will always be something there, non-generalizable method
                        org_list[org_id][list( org_list[org_id])[-1]] += spans_list['text']

                   
    return org_list

In [13]:
textDict_1to5 = openPDFasTextDict(filepath = filepath_1to5)
textDict_full = openPDFasTextDict(filepath = filepath_full)

In [14]:
extractedList_1to5 = extractFromTextDict(textDict_1to5)

In [15]:
extractedList_full = extractFromTextDict(textDict_full)

In [16]:
df_1to5 = pd.DataFrame(extractedList_1to5)

In [17]:
df_full = pd.DataFrame(extractedList_full)

In [18]:
df_1to5.head()

,id,Name,Address,Secteur industriel,Langue de comm,DDD,FAF,N° de télCie,n° de tél,Site Web,...,N° de faxCie,2e n° de tél,Courriel Cie,Fax,2e contact pour,Filiale de,Domaines dintérêt,Projets privilégiés,Nbre de succ,Princip filiales
0,1,3M Canada inc.,"7290, rue Frederick Banting Saint-Laurent QC ...",Équipement et services industriels; Fabricatio...,Français,En tout temps,31 décembre,514-336-5252,800-265-1840,https://www.3mcanada.ca/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A & D Prévost,"305, 12e Avenue Richelieu QC J3L3T2","Matériaux de construction, manufacturiers; Fab...",Français,En tout temps,31 décembre,450-658-8771,NaN,www.adprevost.ca,...,450-658-0077,800-361-4433,info@prevost-architectural.com,450-658-0077,"DonsMadame Marie-Josée Dery, Coordonnatrice Ca...",NaN,NaN,NaN,NaN,NaN
2,3,AAR Aicraft - Services Trois-Rivières,"3750, chemin de l'aéroport Trois-Rivières QC...","Offre des services d'entretien, de réparation ...",Français,En tout temps,31 décembre,819-377-4500,NaN,https://www.aarcorp.com,...,819- 668-8811,NaN,comptabilite@aarcorp.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,ABB (Albert Bob Bob),800 boul. Hymes Saint-Laurent QC H4S0B5,Équipement et services industriels; Fabriquer ...,Français,En tout temps,NaN,514-856-6222,NaN,http://new.abb.com/ca,...,514-856-6297,NaN,NaN,514-856-6297,NaN,Baldor-Dodge-Reliance,NaN,NaN,NaN,NaN
4,5,ABB Canada,800 boul. Hymes Saint-Laurent QC H4S0B5,Commerce de gros et détail; Grossistes-distrib...,Français,En tout temps,31 décembre,438-843-6000,888-856-6266,www.ABB.com/ca,...,514-856-6297,NaN,contact.center@ca.abb.com,514-856-6297,NaN,ABB Bomem,NaN,NaN,NaN,NaN


In [20]:
df_full.head()

,id,Name,Address,Secteur industriel,Langue de comm,DDD,FAF,N° de télCie,n° de tél,Site Web,...,Date approb,Échelledons,Actif,Princip Filiales,Principale filiale,Filiales princip,Principales filiales,Total actif,Principfiliales,ou
0,1,3M Canada inc.,"7290, rue Frederick Banting Saint-Laurent QC ...",Équipement et services industriels; Fabricatio...,Français,En tout temps,31 décembre,514-336-5252,800-265-1840,https://www.3mcanada.ca/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A & D Prévost,"305, 12e Avenue Richelieu QC J3L3T2","Matériaux de construction, manufacturiers; Fab...",Français,En tout temps,31 décembre,450-658-8771,NaN,www.adprevost.ca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,AAR Aicraft - Services Trois-Rivières,"3750, chemin de l'aéroport Trois-Rivières QC...","Offre des services d'entretien, de réparation ...",Français,En tout temps,31 décembre,819-377-4500,NaN,https://www.aarcorp.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,ABB (Albert Bob Bob),800 boul. Hymes Saint-Laurent QC H4S0B5,Équipement et services industriels; Fabriquer ...,Français,En tout temps,NaN,514-856-6222,NaN,http://new.abb.com/ca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,ABB Canada,800 boul. Hymes Saint-Laurent QC H4S0B5,Commerce de gros et détail; Grossistes-distrib...,Français,En tout temps,31 décembre,438-843-6000,888-856-6266,www.ABB.com/ca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [28]:
#Loop through dataframe column pairs
for col1 in df_full.columns:
    for col2 in df_full.columns:
        
        if col1 == col2:
            continue
        
        if similar(col1, col2) >= 0.8:
            print(col1 + " - " + col2 + ": " + str(similar(col1,col2)))

n° de tél - 2e n° de tél: 0.8571428571428571
Domaine dintérêt - Domaines dintérêt: 0.9696969696969697
2e n° de tél - n° de tél: 0.8571428571428571
Domaines dintérêt - Domaine dintérêt: 0.9696969696969697
Princip filiales - Princip Filiales: 0.9375
Princip filiales - Principale filiale: 0.8823529411764706
Princip filiales - Principales filiales: 0.8888888888888888
Princip filiales - Principfiliales: 0.967741935483871
Princip Filiales - Princip filiales: 0.9375
Princip Filiales - Principale filiale: 0.8235294117647058
Princip Filiales - Principales filiales: 0.8333333333333334
Princip Filiales - Principfiliales: 0.9032258064516129
Principale filiale - Princip filiales: 0.8823529411764706
Principale filiale - Princip Filiales: 0.8235294117647058
Principale filiale - Principales filiales: 0.9473684210526315
Principale filiale - Principfiliales: 0.8484848484848485
Principales filiales - Princip filiales: 0.8888888888888888
Principales filiales - Princip Filiales: 0.8333333333333334
Principa